In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
def read_data(path):
    data=pd.read_csv(path)
    print(data.head())
    return data

In [ ]:
application_train=read_data("../input/application_train.csv")
POS_CASH_balance=read_data("../input/POS_CASH_balance.csv")
bureau_balance=read_data("../input/bureau_balance.csv")
previous_application=read_data("../input/previous_application.csv")
installments_payments=read_data("../input/installments_payments.csv")
credit_card_balance=read_data("../input/credit_card_balance.csv")
sample_submission=read_data("../input/sample_submission.csv")
application_test=read_data("../input/application_test.csv")
bureau=read_data("../input/bureau.csv")


In [ ]:
application_train.head()

In [ ]:
application_train.isnull().sum()

In [ ]:
application_train.shape

In [ ]:
target=application_train.corr().TARGET

In [ ]:
target[target>0.0]

In [ ]:
application_train.TARGET.value_counts()

In [ ]:
sns.countplot(application_train.TARGET)

In [ ]:
application_train.head()

In [ ]:
credit_card_balance.head()

In [ ]:
installments_payments.head()

In [ ]:
print(bureau_balance.shape)
bureau_balance.head()

In [ ]:
print(previous_application.shape)
previous_application.head()

In [ ]:
bureau.head()

In [ ]:
previous_application.head()

In [ ]:
previous_application.NAME_CONTRACT_TYPE.value_counts()

## Trying to see if bureau dataset can give us useful information

In [ ]:
bureau.head()

In [ ]:
# Groupby the client id (SK_ID_CURR), count the number of previous loans, and rename the column
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head(20)

In [ ]:
# Join to the training dataframe

application_train = application_train.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

# Fill the missing values with 0 
application_train['previous_loan_counts'] = application_train['previous_loan_counts'].fillna(0)
application_train.head()

In [ ]:
# Checking the correlation
application_train['TARGET'].corr(application_train['previous_loan_counts'])

## Assessing Usefulness of New Variable with r value

To determine if the new variable is useful, we can calculate the Pearson Correlation Coefficient (r-value) between this variable and the target. This measures the strength of a linear relationship between two variables and ranges from -1 (perfectly negatively linear) to +1 (perfectly positively linear). The r-value is not best measure of the "usefulness" of a new variable, but it can give a first approximation of whether a variable will be helpful to a machine learning model. The larger the r-value of a variable with respect to the target, the more a change in this variable is likely to affect the value of the target. Therefore, we look for the variables with the greatest absolute value r-value relative to the target.

We can also visually inspect a relationship with the target using the Kernel Density Estimate (KDE) plot. 

### Kernel Density Estimate Plots

The kernel density estimate plot shows the distribution of a single variable (think of it as a smoothed histogram). To see the different in distributions dependent on the value of a categorical variable, we can color the distributions differently according to the category. For example, we can show the kernel density estimate of the `previous_loan_count` colored by whether the `TARGET` = 1 or 0. The resulting KDE will show any significant differences in the distribution of the variable between people who did not repay their loan (`TARGET == 1`) and the people who did (`TARGET == 0`). This can serve as an indicator of whether a variable will be 'relevant' to a machine learning model. 

We will put this plotting functionality in a function to re-use for any variable. 

In [ ]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [ ]:
installments_payments.head()

In [ ]:
installments_payments.SK_ID_CURR.nunique()

In [ ]:
application_train.shape[0]

In [ ]:
application_test.shape[0]

In [ ]:
bureau